In [2]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 86.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 75.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 KB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.9/507.9 KB 86.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 35.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 48.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 46.5 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import matplotlib.pyplot as plt

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode_outer, regexp_replace
from pyspark.sql.types import DateType
import pandas as pd
import glob
import os

# Inicializar a sessão do Spark com configurações personalizadas
spark_session = SparkSession.builder \
    .appName("Processamento de Dados de Saúde") \
    .master("local[*]") \
    .config("spark.sql.debug.maxToStringFields", "300") \
    .getOrCreate()

# Localizar arquivos JSON no diretório especificado e definir o tamanho do lote
data_files = sorted(glob.glob("minsait/dados/*.json"))
batch_limit = 1000

# Criar DataFrames Pandas vazios para armazenar os dados consolidados
patients_data = pd.DataFrame(columns=["patient_id", "sex", "dob"])
conditions_data = pd.DataFrame(columns=["condition_id", "linked_patient", "condition_name", "date_recorded"])
medications_data = pd.DataFrame(columns=["med_id", "associated_patient", "med_name", "prescribed_date"])

# Função para processar um conjunto de arquivos JSON
def handle_file_group(file_group):
    global patients_data, conditions_data, medications_data

    # Carregar os arquivos JSON em um DataFrame Spark
    raw_data = spark_session.read.option("multiline", "true").json(file_group)

    # Expandir a coluna 'entry' em registros individuais
    expanded_data = raw_data.select(explode_outer(col("entry")).alias("record"))

    # Filtrar e extrair informações de pacientes
    patient_records = expanded_data.filter(col("record.resource.resourceType") == "Patient").select(
        col("record.resource.id").alias("patient_id"),
        col("record.resource.gender").alias("sex"),
        col("record.resource.birthDate").cast(DateType()).alias("dob")
    )

    # Filtrar e extrair informações de condições médicas
    condition_records = expanded_data.filter(col("record.resource.resourceType") == "Condition").select(
        col("record.resource.id").alias("condition_id"),
        regexp_replace(col("record.resource.subject.reference"), "urn:uuid:", "").alias("linked_patient"),
        col("record.resource.code.text").alias("condition_name"),
        col("record.resource.recordedDate").cast(DateType()).alias("date_recorded")
    )

    # Filtrar e extrair informações de prescrições de medicamentos
    medication_records = expanded_data.filter(col("record.resource.resourceType") == "MedicationRequest").select(
        col("record.resource.id").alias("med_id"),
        regexp_replace(col("record.resource.subject.reference"), "urn:uuid:", "").alias("associated_patient"),
        col("record.resource.medicationCodeableConcept.text").alias("med_name"),
        col("record.resource.authoredOn").cast(DateType()).alias("prescribed_date")
    )

    # Transformar os dados Spark em Pandas
    patients_pandas = patient_records.toPandas()
    conditions_pandas = condition_records.toPandas()
    medications_pandas = medication_records.toPandas()

    # Consolidar os dados nos DataFrames globais, eliminando duplicatas
    patients_data = pd.concat([patients_data, patients_pandas]).drop_duplicates(subset=["patient_id"])
    conditions_data = pd.concat([conditions_data, conditions_pandas]).drop_duplicates(subset=["condition_id"])
    medications_data = pd.concat([medications_data, medications_pandas]).drop_duplicates(subset=["med_id"])

# Dividir os arquivos em lotes e processá-los
for batch_start in range(0, len(data_files), batch_limit):
    batch_end = min(batch_start + batch_limit, len(data_files))
    current_batch = data_files[batch_start:batch_end]
    print(f"Executando processamento do lote {batch_start} até {batch_end - 1} (total de arquivos: {len(current_batch)})")
    handle_file_group(current_batch)

# Mostrar um resumo dos dados processados
print("\nInformações dos dados processados:")
print(f"Total de pacientes: {len(patients_data)} registros")
print(f"Total de condições: {len(conditions_data)} registros")
print(f"Total de medicações: {len(medications_data)} registros")

# Opcional: Exportar os dados para arquivos CSV
patients_data.to_csv("patients_output.csv", index=False)
conditions_data.to_csv("conditions_output.csv", index=False)
medications_data.to_csv("medications_output.csv", index=False)


Executando processamento do lote 0 até 999 (total de arquivos: 1000)


Executando processamento do lote 1000 até 1179 (total de arquivos: 180)


[Stage 11:====================================================>   (15 + 1) / 16]


Informações dos dados processados:
Total de pacientes: 1180 registros
Total de condições: 8766 registros
Total de medicações: 14102 registros


In [7]:
# Análise 1: Quais são as 10 condições médicas mais comuns?
top_conditions = conditions_data["condition_name"].value_counts().head(10)
print("\nAs 10 condições médicas mais comuns:")
print(top_conditions)

# Plotagem das 10 condições mais comuns
plt.figure(figsize=(12, 6))
top_conditions.plot(kind="bar")
plt.title("10 Condições Médicas Mais Comuns")
plt.xlabel("Condição")
plt.ylabel("Número de Ocorrências")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("top_10_conditions.png")
plt.close()

# Análise 2: Quais são os 10 medicamentos mais prescritos?
top_medications = medications_data["med_name"].value_counts().head(10)
print("\nOs 10 medicamentos mais prescritos:")
print(top_medications)

# Plotagem dos 10 medicamentos mais prescritos
plt.figure(figsize=(12, 6))
top_medications.plot(kind="bar")
plt.title("10 Medicamentos Mais Prescritos")
plt.xlabel("Medicamento")
plt.ylabel("Número de Prescrições")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("top_10_medications.png")
plt.close()

# Análise 3: Quantos pacientes são do sexo masculino?
male_patients = patients_data[patients_data["sex"] == "male"].shape[0]
total_patients = patients_data.shape[0]
female_patients = total_patients - male_patients
print(f"\nNúmero de pacientes do sexo masculino: {male_patients}")
print(f"Número total de pacientes: {total_patients}")

# Plotagem da distribuição de gênero
plt.figure(figsize=(8, 6))
plt.bar(["Masculino", "Feminino"], [male_patients, female_patients])
plt.title("Distribuição de Pacientes por Gênero")
plt.ylabel("Número de Pacientes")
for i, v in enumerate([male_patients, female_patients]):
    plt.text(i, v + 0.5, str(v), ha="center")
plt.tight_layout()
plt.savefig("gender_distribution.png")
plt.close()


As 10 condições médicas mais comuns:
condition_name
Viral sinusitis (disorder)                 1237
Acute viral pharyngitis (disorder)          754
Acute bronchitis (disorder)                 548
Normal pregnancy                            516
Prediabetes                                 388
Anemia (disorder)                           382
Body mass index 30+ - obesity (finding)     346
Hypertension                                330
Chronic sinusitis (disorder)                251
Miscarriage in first trimester              208
Name: count, dtype: int64

Os 10 medicamentos mais prescritos:
med_name
1 ML Epoetin Alfa 4000 UNT/ML Injection [Epogen]       4133
Simvistatin 10 MG                                      2306
Cisplatin 50 MG Injection                               880
PACLitaxel 100 MG Injection                             820
Acetaminophen 325 MG Oral Tablet                        622
Amoxicillin 250 MG / Clavulanate 125 MG Oral Tablet     266
Naproxen sodium 220 MG Oral Tablet 

In [28]:
import duckdb

# Conectar ao DuckDB e criar/salvar tabelas
con = duckdb.connect('/opt/metabaseduck/health_data.duckdb')  # Cria ou conecta a um arquivo DuckDB

# Salvar os DataFrames como tabelas no DuckDB
con.execute("CREATE OR REPLACE TABLE patients_table AS SELECT * FROM patients_data")
con.execute("CREATE OR REPLACE TABLE conditions_table AS SELECT * FROM conditions_data")
con.execute("CREATE OR REPLACE TABLE medication_table AS SELECT * FROM medications_data")

# Análise 1: Quais são as 10 condições médicas mais comuns?
top_conditions_query = """
SELECT condition_name, COUNT(*) as count
FROM conditions_table
GROUP BY condition_name
ORDER BY count DESC
LIMIT 10
"""
top_conditions = con.execute(top_conditions_query).fetchdf()
print("\nAs 10 condições médicas mais comuns:")
print(top_conditions)

# Plotagem das 10 condições mais comuns
plt.figure(figsize=(12, 6))
plt.bar(top_conditions['condition_name'], top_conditions['count'])
plt.title('10 Condições Médicas Mais Comuns')
plt.xlabel('Condição')
plt.ylabel('Número de Ocorrências')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('top_10_conditions.png')
plt.close()

# Análise 2: Quais são os 10 medicamentos mais prescritos?
top_medications_query = """
SELECT med_name, COUNT(*) as count
FROM medication_table
GROUP BY med_name
ORDER BY count DESC
LIMIT 10
"""
top_medications = con.execute(top_medications_query).fetchdf()
print("\nOs 10 medicamentos mais prescritos:")
print(top_medications)

# Plotagem dos 10 medicamentos mais prescritos
plt.figure(figsize=(12, 6))
plt.bar(top_medications['med_name'], top_medications['count'])
plt.title('10 Medicamentos Mais Prescritos')
plt.xlabel('Medicamento')
plt.ylabel('Número de Prescrições')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('top_10_medications.png')
plt.close()

# Análise 3: Quantos pacientes são do sexo masculino?
gender_query = """
SELECT sex, COUNT(*) as count
FROM patients_table
GROUP BY sex
"""
gender_distribution = con.execute(gender_query).fetchdf()
male_patients = gender_distribution[gender_distribution['sex'] == 'male']['count'].iloc[0] if 'male' in gender_distribution['sex'].values else 0
total_patients = gender_distribution['count'].sum()
female_patients = total_patients - male_patients
print(f"\nNúmero de pacientes do sexo masculino: {male_patients}")
print(f"Número total de pacientes: {total_patients}")

# Plotagem da distribuição de gênero
plt.figure(figsize=(8, 6))
plt.bar(['Masculino', 'Feminino'], [male_patients, female_patients])
plt.title('Distribuição de Pacientes por Gênero')
plt.ylabel('Número de Pacientes')
for i, v in enumerate([male_patients, female_patients]):
    plt.text(i, v + 0.5, str(v), ha='center')
plt.tight_layout()
plt.savefig('gender_distribution.png')
plt.close()


As 10 condições médicas mais comuns:
                            condition_name  count
0               Viral sinusitis (disorder)   1237
1       Acute viral pharyngitis (disorder)    754
2              Acute bronchitis (disorder)    548
3                         Normal pregnancy    516
4                              Prediabetes    388
5                        Anemia (disorder)    382
6  Body mass index 30+ - obesity (finding)    346
7                             Hypertension    330
8             Chronic sinusitis (disorder)    251
9           Miscarriage in first trimester    208

Os 10 medicamentos mais prescritos:
                                            med_name  count
0   1 ML Epoetin Alfa 4000 UNT/ML Injection [Epogen]   4133
1                                  Simvistatin 10 MG   2306
2                          Cisplatin 50 MG Injection    880
3                        PACLitaxel 100 MG Injection    820
4                   Acetaminophen 325 MG Oral Tablet    622
5  Amoxicillin 

In [30]:
import duckdb
con = duckdb.connect('/opt/metabaseduck/health_data.duckdb')
print(con.execute("SHOW TABLES").fetchall())
con.close()

[('conditions_table',), ('medication_table',), ('patients_table',)]
